# Simulation of M9 medium with and without Casamino acids

In [2]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd
import numpy as np

In [19]:
def enhanced_growth(model, base_medium, new_metabs):
    local_medium = base_medium
    with model:
        medium = model.medium
        for met in new_metabs:
            local_medium[met] = 10.0
        model.medium = local_medium
        sol = model.optimize()
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate with additives: ' + str(round(base_growth, 4)))
    print('Doubling time with additives: ' + str(round(dt, 2)) + ' min')

def find_additives(model, base_medium):
    with model:
        medium = model.medium
        model.medium = base_medium
        sol = model.optimize()      
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate without additives: ' + str(round(base_growth, 4)))
    print('Doubling time without additives: ' + str(round(dt, 2)) + ' min')
        
    enhancement = {}
    for ex in list(model.exchanges):
        if ex.id not in base_medium.keys():
            with model:
                medium = model.medium
                base_medium.update({ex.id:10.0})
                model.medium = base_medium
                sol = model.optimize()
                if sol.objective_value > base_growth:
                    enhancement[ex.id] = sol.objective_value - base_growth
                base_medium.pop(ex.id)
    
    enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff']).sort_values(by=['diff'], ascending=False)
    
    return enh

In [37]:
CasA = rg.growth.load_all_media_from_db('../../refinegems/data/media_db.csv')[9]['BiGG_EX'].to_list()
M9 = rg.growth.load_all_media_from_db('../../refinegems/data/media_db.csv')[2]['BiGG_EX'].to_list()
CasA_med = {i: 10.0 for i in CasA}
M9_med = {i: 10.0 for i in M9}
if (M9_med['EX_o2_e'] == 10.0):
    M9_med['EX_o2_e'] = 20.0
M9_med.pop('EX_ni2_e') # missing in the models

# from simulation with refinegems
M9_med['EX_ala_B_e'] = 10.0
M9_med['EX_cgly_e'] = 10.0

In [41]:
model = rg.load.load_model_cobra('../models/Cstr_15.xml')

In [42]:
find_additives(model, M9_med)

Growth rate without additives: 0.639
Doubling time without additives: 65.08 min


,exchange,diff
51,EX_malthp_e,2.327783e+00
87,EX_sucr_e,5.477137e-01
93,EX_tre_e,5.477137e-01
50,EX_malt_e,5.477137e-01
44,EX_lcts_e,4.450174e-01
...,...,...
56,EX_mnl1p_e,7.327472e-15
29,EX_gly_asp__L_e,6.328271e-15
21,EX_fum_e,4.107825e-15
36,EX_h2s_e,3.663736e-15


In [43]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.639
Doubling time with additives: 65.08 min
